# Lab J1 — Matin : NumPy & Pandas
**Formation IA, Deep Learning & Machine Learning** — Julien Rolland  
**Public :** M2 Développement Fullstack

---

## Objectifs

- Manipuler des tableaux NumPy (création, slicing, reshape, broadcasting)
- Mesurer concrètement le gain de la vectorisation vs les boucles `for`
- Charger, explorer et nettoyer un dataset avec Pandas
- **Implémenter des couches neuronales de base en NumPy pur**

---

In [1]:
import numpy as np
import pandas as pd
import time

print('NumPy  :', np.__version__)
print('Pandas :', pd.__version__)

NumPy  : 2.4.2
Pandas : 2.3.3


---
## Partie 1 — NumPy

### 1.1 Créer des tableaux

In [2]:
v = np.array([1, 2, 3, 4, 5])
print('vecteur      :', v)

M = np.array([[1, 2, 3],
              [4, 5, 6]])
print('matrice :\n', M)

vecteur      : [1 2 3 4 5]
matrice :
 [[1 2 3]
 [4 5 6]]


In [3]:
print(np.zeros((3, 4)))          # matrice de zeros
print(np.ones((2, 3)))           # matrice de uns
print(np.eye(3))                 # matrice identite
print(np.arange(0, 10, 2))
print(np.linspace(0, 1, 5))     # 5 points regulierement espaces
print(np.random.randn(2, 3))    # valeurs gaussiennes N(0,1)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[1. 1. 1.]
 [1. 1. 1.]]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[0 2 4 6 8]
[0.   0.25 0.5  0.75 1.  ]
[[-0.25738275  0.34061673 -1.26823152]
 [-1.62560767 -0.64906903  0.76033832]]


### 1.2 Shape, rank, dtype

In [4]:
X = np.random.randn(50, 4)   # 50 exemples, 4 features

print('shape  :', X.shape)
print('ndim   :', X.ndim)
print('dtype  :', X.dtype)
print('size   :', X.size)

shape  : (50, 4)
ndim   : 2
dtype  : float64
size   : 200


In [5]:
X_int = X.astype(np.int32)
print('dtype apres conversion :', X_int.dtype)

# Un tableau 3D : batch de 8 images 28x28
images = np.random.randint(0, 256, size=(8, 28, 28), dtype=np.uint8)
print('shape images :', images.shape)
print('ndim         :', images.ndim)

dtype apres conversion : int32
shape images : (8, 28, 28)
ndim         : 3


### 1.3 Boucle `for` vs vectorisation

On mesure le temps d'exécution sur un produit scalaire de taille $n = 10^6$.

In [6]:
n = 1_000_000
u = list(range(n))
v = list(range(n))

def dot_loop(u, v):
    result = 0.0
    for u_i, v_i in zip(u, v):
        result += u_i * v_i
    return result

t0 = time.perf_counter()
res_loop = dot_loop(u, v)
t_loop = time.perf_counter() - t0

print(f'Boucle for  : {t_loop*1000:.1f} ms  (resultat = {res_loop:.2e})')

Boucle for  : 74.4 ms  (resultat = 3.33e+17)


In [7]:
u_np = np.array(u)
v_np = np.array(v)

t0 = time.perf_counter()
res_np = np.dot(u_np, v_np)
t_np = time.perf_counter() - t0

print(f'np.dot      : {t_np*1000:.2f} ms  (resultat = {res_np:.2e})')
print(f'Acceleration: x{t_loop/t_np:.0f}')

np.dot      : 1.00 ms  (resultat = 3.33e+17)
Acceleration: x75


### 1.4 Slicing avancé

In [8]:
X = np.arange(12).reshape(3, 4)
print('X :\n', X)
print()

print('Ligne 0         :', X[0])
print('Colonne 2       :', X[:, 2])
print('Sous-matrice :\n', X[1:, 1:3])    # lignes 1-2, cols 1-2
print('Lignes paires :\n', X[::2])

X :
 [[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]

Ligne 0         : [0 1 2 3]
Colonne 2       : [ 2  6 10]
Sous-matrice :
 [[ 5  6]
 [ 9 10]]
Lignes paires :
 [[ 0  1  2  3]
 [ 8  9 10 11]]


In [9]:
# Indexation booleenne
x = np.array([3, -1, 7, -4, 2, -5])

mask = x > 0
print('masque   :', mask)
print('positifs :', x[mask])

# Remplacer les negatifs par 0 (relu !)
x_relu = x.copy()
x_relu[x_relu < 0] = 0
print('relu     :', x_relu)

masque   : [ True False  True False  True False]
positifs : [3 7 2]
relu     : [3 0 7 0 2 0]


### 1.5 Reshape & vues

> **Important :** `reshape` retourne une **vue** — pas une copie.  
> Modifier la vue modifie le tableau original.

In [10]:
v = np.arange(6)
print('original  :', v)

M = v.reshape(2, 3)
print('reshape(2,3) :\n', M)

# -1 = numpy calcule la dimension automatiquement
print('reshape(3,-1) :\n', v.reshape(3, -1))

# Attention : vue !
M[0, 0] = 99
print('v apres modification de M :', v)  # v[0] est aussi 99 !

original  : [0 1 2 3 4 5]
reshape(2,3) :
 [[0 1 2]
 [3 4 5]]
reshape(3,-1) :
 [[0 1]
 [2 3]
 [4 5]]
v apres modification de M : [99  1  2  3  4  5]


In [11]:
v = np.arange(6)
M_copy = v.reshape(2, 3).copy()
M_copy[0, 0] = 99
print('v inchange :', v)

v inchange : [0 1 2 3 4 5]


### 1.6 Broadcasting

NumPy aligne les shapes **depuis la droite** et étire automatiquement les dimensions de taille 1.

In [12]:
X = np.ones((4, 3))           # shape (4, 3)
b = np.array([10, 20, 30])    # shape    (3,)

print('X + b :\n', X + b)     # (4,3) + (3,) => (4,3)

X + b :
 [[11. 21. 31.]
 [11. 21. 31.]
 [11. 21. 31.]
 [11. 21. 31.]]


In [13]:
np.random.seed(42)
X = np.random.randn(100, 5)

mean = X.mean(axis=0)         # shape (5,)  -- moyenne par feature
std  = X.std(axis=0)          # shape (5,)
X_norm = (X - mean) / std     # broadcasting : (100,5) op (5,) => (100,5)

print('Mean apres normalisation :', X_norm.mean(axis=0).round(10))
print('Std  apres normalisation :', X_norm.std(axis=0).round(10))

Mean apres normalisation : [-0. -0. -0. -0.  0.]
Std  apres normalisation : [1. 1. 1. 1. 1.]


In [14]:
points = np.array([1.0, 3.0, 6.0, 10.0])

# col (4,1) - ligne (1,4) => matrice (4,4)
dist = np.abs(points[:, np.newaxis] - points[np.newaxis, :])
print('Matrice de distances :\n', dist)

Matrice de distances :
 [[0. 2. 5. 9.]
 [2. 0. 3. 7.]
 [5. 3. 0. 4.]
 [9. 7. 4. 0.]]


### 1.7 Opérations d'agrégation utiles

In [15]:
X = np.array([[1, 5, 3],
              [4, 2, 6]])

print('sum total     :', X.sum())
print('sum par col   :', X.sum(axis=0))
print('sum par ligne :', X.sum(axis=1))
print('max total     :', X.max())
print('argmax        :', X.argmax())        # indice dans le tableau aplati
print('argmax par col:', X.argmax(axis=0))  # ligne du max pour chaque colonne
print('mean          :', X.mean())
print('std           :', X.std().round(4))

sum total     : 21
sum par col   : [5 7 9]
sum par ligne : [ 9 12]
max total     : 6
argmax        : 5
argmax par col: [1 0 1]
mean          : 3.5
std           : 1.7078


### 1.8 Produit matriciel avec `@`

L'opérateur `@` est le raccourci Python pour `np.matmul`. C'est l'opération centrale de tout réseau de neurones.

In [16]:
A = np.array([[1, 2],
              [3, 4]])
B = np.array([[5, 6],
              [7, 8]])

print('A @ B :\n', A @ B)
print('equivalent np.matmul :', np.allclose(A @ B, np.matmul(A, B)))

A @ B :
 [[19 22]
 [43 50]]
equivalent np.matmul : True


In [17]:
# Forward pass d'une couche lineaire sans biais
# X (m, n_in)  @ W.T (n_in, n_out)  =>  (m, n_out)
np.random.seed(0)
X = np.random.randn(8, 3)    # batch de 8 exemples, 3 features
W = np.random.randn(5, 3)    # 5 neurones de sortie

out = X @ W.T
print('X shape  :', X.shape)
print('W shape  :', W.shape)
print('out shape:', out.shape)

X shape  : (8, 3)
W shape  : (5, 3)
out shape: (8, 5)


In [18]:
# Chaine de couches : X -> H1 -> H2
X  = np.random.randn(8, 4)
W1 = np.random.randn(6, 4)   # couche 1 : 4 -> 6
W2 = np.random.randn(3, 6)   # couche 2 : 6 -> 3

H1 = X  @ W1.T   # (8, 6)
H2 = H1 @ W2.T   # (8, 3)

print('H1 shape :', H1.shape)
print('H2 shape :', H2.shape)

H1 shape : (8, 6)
H2 shape : (8, 3)


---
## Partie 2 — Pandas

### 2.1 Créer un dataset synthétique

In [19]:
np.random.seed(0)
n = 20
df = pd.DataFrame({
    'age'      : np.random.randint(22, 45, n).astype(float),
    'exp_ans'  : np.random.randint(0, 15, n).astype(float),
    'salaire'  : np.random.randint(35000, 90000, n).astype(float),
    'ville'    : np.random.choice(['Paris', 'Lyon', 'Marseille'], n),
    'embauche' : np.random.choice([0, 1], n),
})
df.loc[[2, 7, 13], 'age']     = np.nan
df.loc[[4, 11],    'salaire'] = np.nan
df.loc[[6],        'ville']   = np.nan
df.head(10)

,age,exp_ans,salaire,ville,embauche
0,34.0,9.0,59999.0,Paris,0
1,37.0,13.0,51321.0,Lyon,0
2,NaN,8.0,87489.0,Marseille,1
3,22.0,9.0,54129.0,Marseille,1
4,25.0,4.0,NaN,Paris,0
5,25.0,3.0,76504.0,Lyon,1
6,29.0,0.0,84866.0,NaN,0
7,NaN,3.0,53676.0,Lyon,0
8,41.0,5.0,66230.0,Lyon,0
9,43.0,14.0,46723.0,Marseille,0


### 2.2 Exploration

In [20]:
print('Shape :', df.shape)
print()
df.info()

Shape : (20, 5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       17 non-null     float64
 1   exp_ans   20 non-null     float64
 2   salaire   18 non-null     float64
 3   ville     19 non-null     object 
 4   embauche  20 non-null     int64  
dtypes: float64(3), int64(1), object(1)
memory usage: 932.0+ bytes


In [21]:
df.describe()

,age,exp_ans,salaire,embauche
count,17.000000,20.000000,18.00000,20.000000
mean,31.294118,5.900000,62183.00000,0.500000
std,6.807759,4.722845,14344.08267,0.512989
min,22.000000,0.000000,38560.00000,0.000000
25%,26.000000,3.000000,53789.25000,0.000000
50%,29.000000,3.500000,60165.50000,0.500000
75%,37.000000,9.000000,73935.50000,1.000000
max,43.000000,14.000000,87489.00000,1.000000


In [22]:
print('NaN par colonne :')
print(df.isna().sum())
print()
print('Pourcentage NaN :')
print((df.isna().sum() / len(df) * 100).round(1))

NaN par colonne :
age         3
exp_ans     0
salaire     2
ville       1
embauche    0
dtype: int64

Pourcentage NaN :
age         15.0
exp_ans      0.0
salaire     10.0
ville        5.0
embauche     0.0
dtype: float64


In [23]:
print('Candidats embauchés :', df[df['embauche'] == 1].shape[0])
print()
print('Répartition par ville :')
print(df['ville'].value_counts())
print()
print('Salaire moyen par ville :')
print(df.groupby('ville')['salaire'].mean().round(0))

Candidats embauchés : 10

Répartition par ville :
ville
Marseille    7
Paris        6
Lyon         6
Name: count, dtype: int64

Salaire moyen par ville :
ville
Lyon         60224.0
Marseille    61257.0
Paris        61109.0
Name: salaire, dtype: float64


### 2.3 Gestion des valeurs manquantes (NaN)

In [24]:
df_clean = df.copy()

# Strategie 1 : imputer par la moyenne/mediane (numeriques)
df_clean['age']     = df_clean['age'].fillna(df_clean['age'].mean())
df_clean['salaire'] = df_clean['salaire'].fillna(df_clean['salaire'].median())

# Strategie 2 : valeur la plus frequente (categoriel)
mode_ville = df_clean['ville'].mode()[0]
df_clean['ville'] = df_clean['ville'].fillna(mode_ville)

print('NaN restants :', df_clean.isna().sum().sum())
df_clean.head(10)

NaN restants : 0


,age,exp_ans,salaire,ville,embauche
0,34.000000,9.0,59999.0,Paris,0
1,37.000000,13.0,51321.0,Lyon,0
2,31.294118,8.0,87489.0,Marseille,1
3,22.000000,9.0,54129.0,Marseille,1
4,25.000000,4.0,60165.5,Paris,0
5,25.000000,3.0,76504.0,Lyon,1
6,29.000000,0.0,84866.0,Marseille,0
7,31.294118,3.0,53676.0,Lyon,0
8,41.000000,5.0,66230.0,Lyon,0
9,43.000000,14.0,46723.0,Marseille,0


In [25]:
# Strategie 3 (alternative) : supprimer les lignes avec NaN
df_dropped = df.dropna()
print(f'Lignes apres dropna : {len(df_dropped)} / {len(df)}')

Lignes apres dropna : 14 / 20


### 2.4 One-Hot Encoding

In [26]:
df_encoded = pd.get_dummies(df_clean, columns=['ville'], dtype=float)
print('Colonnes apres encodage :', df_encoded.columns.tolist())
df_encoded.head(5)

Colonnes apres encodage : ['age', 'exp_ans', 'salaire', 'embauche', 'ville_Lyon', 'ville_Marseille', 'ville_Paris']


,age,exp_ans,salaire,embauche,ville_Lyon,ville_Marseille,ville_Paris
0,34.000000,9.0,59999.0,0,0.0,0.0,1.0
1,37.000000,13.0,51321.0,0,1.0,0.0,0.0
2,31.294118,8.0,87489.0,1,0.0,1.0,0.0
3,22.000000,9.0,54129.0,1,0.0,1.0,0.0
4,25.000000,4.0,60165.5,0,0.0,0.0,1.0


In [27]:
# drop_first=True pour eviter la colinearite (dummy variable trap)
df_encoded = pd.get_dummies(df_clean, columns=['ville'],
                             drop_first=True, dtype=float)
print('Colonnes (drop_first) :', df_encoded.columns.tolist())

Colonnes (drop_first) : ['age', 'exp_ans', 'salaire', 'embauche', 'ville_Marseille', 'ville_Paris']


### 2.5 Passage à NumPy — prêt pour le modèle

In [28]:
feature_cols = ['age', 'exp_ans', 'salaire', 'ville_Marseille', 'ville_Paris']
target_col   = 'embauche'

X = df_encoded[feature_cols].values
y = df_encoded[target_col].values

print('X shape :', X.shape)
print('y shape :', y.shape)
print('X dtype :', X.dtype)
print('y dtype :', y.dtype)

X shape : (20, 5)
y shape : (20,)
X dtype : float64
y dtype : int64


In [29]:
mean = X.mean(axis=0)
std  = X.std(axis=0)
X_norm = (X - mean) / std

print('Mean (doit etre ~0) :', X_norm.mean(axis=0).round(10))
print('Std  (doit etre ~1) :', X_norm.std(axis=0).round(10))

Mean (doit etre ~0) : [ 0. -0.  0. -0. -0.]
Std  (doit etre ~1) : [1. 1. 1. 1. 1.]


---
## Récapitulatif

| Étape | Outil | Ce qu'on a fait |
|---|---|---|
| Création | `np.array`, `np.random` | Tableaux de différents rangs et dtypes |
| Vectorisation | `np.dot` | ~100× plus rapide que `zip` + boucle |
| Slicing | `X[1:, ::2]`, masques booléens | Extraction de sous-tableaux, filtre ReLU |
| Broadcasting | `X - mean` | Normalisation sans boucle |
| Produit matriciel | `@` | Forward pass d'une couche linéaire |
| Exploration | `df.info()`, `df.describe()` | Comprendre la structure des données |
| Nettoyage | `fillna`, `dropna` | Traitement des NaN |
| Encodage | `get_dummies` | Variables catégorielles → binaires |
| Export | `.values` | DataFrame → ndarray pour le modèle |

---
## Partie 3 — Exercices : Couches Neuronales en NumPy

Implémenter les briques de base d'un réseau de neurones **en NumPy pur**, sans PyTorch.  
Chaque fonction doit être **vectorisée** (pas de boucle `for` sur les éléments).

### Exercice 1 — ReLU

$$\text{ReLU}(x) = \max(0,\, x)$$

Appliquée élément par élément sur un tableau de shape quelconque.

In [30]:
def relu(x: np.ndarray) -> np.ndarray:
    return np.maximum(0, x)

In [31]:
x = np.array([-3., -1., 0., 2., 5.])
out = relu(x)

assert out.shape == x.shape
assert np.allclose(out, [0., 0., 0., 2., 5.])
assert (out >= 0).all()

X = np.array([[-1., 2.], [3., -4.]])
assert relu(X).shape == X.shape
assert np.allclose(relu(X), [[0., 2.], [3., 0.]])

print('ReLU OK :', out)

ReLU OK : [0. 0. 0. 2. 5.]


### Exercice 2 — Sigmoid

$$\sigma(x) = \frac{1}{1 + e^{-x}}$$

Sortie dans $]0, 1[$ — utilisée en classification binaire.

In [32]:
def sigmoid(x: np.ndarray) -> np.ndarray:
    return 1.0 / (1.0 + np.exp(-x))

In [33]:
x = np.array([-10., -2., 0., 2., 10.])
out = sigmoid(x)

assert out.shape == x.shape
assert np.allclose(out[2], 0.5)
assert ((out > 0) & (out < 1)).all()
assert out[0] < 0.001
assert out[-1] > 0.999

print('Sigmoid OK :', out.round(4))

Sigmoid OK : [0.     0.1192 0.5    0.8808 1.    ]


### Exercice 3 — Softmax (avec exp-trick)

$$\text{softmax}(x_i) = \frac{e^{x_i}}{\sum_j e^{x_j}}$$

**Problème numérique :** si $x_i$ est grand, $e^{x_i}$ explose (`inf`).  
**Exp-trick :** soustraire $c = \max_j x_j$ avant l'exponentielle — ne change pas le résultat :

$$\text{softmax}(x_i) = \frac{e^{x_i - c}}{\sum_j e^{x_j - c}}$$

Entrée : `(batch, n_classes)` — Sortie : même shape.

In [34]:
def softmax(x: np.ndarray) -> np.ndarray:
    c = x.max(axis=-1, keepdims=True)   # exp-trick
    e = np.exp(x - c)
    return e / e.sum(axis=-1, keepdims=True)

In [35]:
x = np.array([1., 2., 3.])
out = softmax(x)
assert out.shape == x.shape
assert np.allclose(out.sum(), 1.0)
assert (out > 0).all()

X = np.array([[1., 2., 3., 4.], [1., 1., 1., 1.]])
out_batch = softmax(X)
assert out_batch.shape == X.shape
assert np.allclose(out_batch.sum(axis=-1), 1.0)

# exp-trick : pas de NaN meme avec des valeurs extremes
x_large = np.array([1000., 1001., 1002.])
out_large = softmax(x_large)
assert not np.any(np.isnan(out_large))
assert np.allclose(out_large.sum(), 1.0)

print('Softmax OK :', out.round(4))
print('Batch      :', out_batch.round(4))

Softmax OK : [0.09   0.2447 0.6652]
Batch      : [[0.0321 0.0871 0.2369 0.6439]
 [0.25   0.25   0.25   0.25  ]]


### Exercice 4 — Couche Linéaire (forward)

$$\text{Linear}(X) = X W^\top + b$$

avec $X \in \mathbb{R}^{m \times n_{\text{in}}}$, $W \in \mathbb{R}^{n_{\text{out}} \times n_{\text{in}}}$, $b \in \mathbb{R}^{n_{\text{out}}}$

Résultat : $\mathbb{R}^{m \times n_{\text{out}}}$

In [36]:
def linear(X: np.ndarray, W: np.ndarray, b: np.ndarray) -> np.ndarray:
    return X @ W.T + b

In [37]:
np.random.seed(1)
m, n_in, n_out = 8, 4, 6

X = np.random.randn(m, n_in)
W = np.random.randn(n_out, n_in)
b = np.zeros(n_out)

out = linear(X, W, b)
assert out.shape == (m, n_out)
assert np.allclose(out, X @ W.T + b)

b2 = np.ones(n_out) * 5.0
out2 = linear(X, W, b2)
assert np.allclose(out2, out + 5.0)

print('Linear OK — shape :', out.shape)

Linear OK — shape : (8, 6)


### Exercice 5 — Convolution 1D

La convolution 1D fait glisser un filtre $k$ de taille $K$ sur un signal $x$ de taille $N$ :

$$(x * k)_i = \sum_{j=0}^{K-1} x_{i+j} \cdot k_j, \quad i \in [0,\, N - K]$$

Taille de sortie : $N - K + 1$ (pas de padding, stride = 1).

In [38]:
def conv1d(x: np.ndarray, kernel: np.ndarray) -> np.ndarray:
    N, K = len(x), len(kernel)
    out_size = N - K + 1
    # chaque position i = produit scalaire entre x[i:i+K] et kernel
    return np.array([np.dot(x[i:i + K], kernel) for i in range(out_size)])

In [39]:
x      = np.array([1., 2., 3., 4., 5.])
kernel = np.array([1., 0., -1.])          # detecteur de gradient
out    = conv1d(x, kernel)

assert out.shape == (len(x) - len(kernel) + 1,)
assert np.allclose(out, [-2., -2., -2.])

# Filtre moyenne mobile
k_avg   = np.array([1/3, 1/3, 1/3])
out_avg = conv1d(x, k_avg)
assert out_avg.shape == (3,)
assert np.allclose(out_avg, [2., 3., 4.])

print('Conv1D OK :', out)
print('Moyenne mobile :', out_avg)

Conv1D OK : [-2. -2. -2.]
Moyenne mobile : [2. 3. 4.]
